In [ ]:
!pip install unstructured -q
!pip install unstructured[local-inference] -q
!pip install chromadb==0.4.22 -q
!pip install bitsandbytes==0.41.1 langchain==0.1.1 transformers==4.37.0 -q
!pip install accelerate==0.24 -q
!pip install sentence_transformers -q
!pip install huggingface_hub -q

In [ ]:
!pip install gradio

In [ ]:
pip install transformers==4.42.3

  Using cached transformers-4.42.3-py3-none-any.whl (9.3 MB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.0
    Uninstalling transformers-4.37.0:
      Successfully uninstalled transformers-4.37.0


In [ ]:
#import streamlit as st
import csv
import sys
import torch
import transformers
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
)
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings, HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory




In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read).

In [ ]:
use_4bit=True
use_nested_quant=True
bnb_4bit_quant_type="nf4"
compute_dtype="float16"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [ ]:
import transformers
model_name="mistralai/Mistral-7B-Instruct-v0.2"
model_config=transformers.AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [ ]:
pipeline = transformers.pipeline("text-generation",
                                 model = model,
                                 tokenizer = tokenizer,
                                 torch_dtype = torch.bfloat16,
                                 device_map = "auto",
                                 max_new_tokens = 512,
                                 min_new_tokens = -1,
                                 temperature = 0.01,
                                 do_sample = True,
                                 top_k = 30,
                                 num_return_sequences =1,
                                 eos_token_id = tokenizer.eos_token_id)

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline)

In [ ]:
title_template = PromptTemplate(input_variables = ['topic'],
                                template = 'Write me 1 Youtube video title about {topic}')

script_template = PromptTemplate(input_variables = ['title'],
                                template = 'Write me Youtube script based on the title {title}')



In [ ]:
from langchain.chains import LLMChain, SimpleSequentialChain

title_chain = LLMChain(llm = llm, prompt=title_template)
script_chain = LLMChain(llm = llm, prompt= script_template)

sequential_chain = SimpleSequentialChain(chains = [title_chain, script_chain])

In [ ]:
def generate_response(prompt):
    if prompt:
        response = sequential_chain.run({prompt})
        return response
    return "Please enter a prompt."


In [ ]:
import gradio as gr
iface = gr.Interface(
    fn=generate_response,
    inputs=gr.Textbox(lines=2, placeholder="Plug in your prompt here"),
    outputs="text",
    title="YouTube Video Script Generator with LangChain 🦜🔗",
    description="Enter a prompt to generate a YouTube video script."
)

# Launch the Gradio interface
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3f2187c01377af99c6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
